# Algoritmo de $K$-medias

## Un ejemplito

Consideramos un conjunto de datos el cuál queremos categorizar en dos grupos:

In [ ]:
D = ((1, 3),
     (2, 3),
     (5, 3),
     (7, 3))

Consideramos dos grupos iniciales representados por sus centros:

In [ ]:
c1 = [4, 5]
c2 = [6, 1]

Veamos cómo graficar esta información. Utilizamos el módulo `pyplot` de la biblioteca `matplotlib`.

In [ ]:
import matplotlib.pyplot as plt

# creamos una figura (es como un lienzo vacío)
fig = plt.figure()

# creamos ejes sobre la figura (nos relaciona datos con coordenadas gráficas)
ax = fig.add_subplot()

# creamos una gráfica de puntos, llamada scatter plot en inglés
# le debemos pasar dos secuencias de coordenadas y le pedimos que dibuje
# los puntos con color negro
ax.scatter([x for x, _ in D], [y for _, y in D], c = "black")

# creamos otra gráfica de puntos para los centros, los cuáles pintamos
# con color azúl
ax.scatter([c1[0], c2[0]], [c1[1], c2[1]], c = "blue")

fig;

El primer paso es calcular la distancia de cada ejemplo de los datos a cada centro, el grupo asignado a un ejemplo corresponde al del centro cuya distancia es menor.

In [ ]:
def distance_squared(p1, p2):
    return sum((x1 - x2)**2 for x1, x2 in zip(p1, p2))

Veamos cómo nos quedan las asignaciones:

In [ ]:
ass = []
for p in D:
    d1 = distance_squared(p, c1)
    d2 = distance_squared(p, c2)
    print(f"El punto {p} está a distancia {d1} de c1 y a distancia {d2} de c2")
    g = min([(d1, 1), (d2, 2)])[1]
    print(f"El grupo asignado a {p} es {g}")
    print()
    ass.append(g)

Ahora vamos a mejorar los centros a partir de las asignaciones, el nuevo centro corresponde al punto medio entre los puntos asignados a su grupo correspondiente.

In [ ]:
n1 = ass.count(1)
c1 = [
    sum(x for i, (x, _) in enumerate(D) if ass[i] == 1) / n1,
    sum(x for i, (_, x) in enumerate(D) if ass[i] == 1) / n1,
]

n2 = ass.count(2)
c2 = [
    sum(x for i, (x, _) in enumerate(D) if ass[i] == 2) / n2,
    sum(x for i, (_, x) in enumerate(D) if ass[i] == 2) / n2,
]

c1, c2

Ahora volvemos a repetir el proceso, ¿qué asignaciones obtenemos al considerar estos centros?

In [ ]:
ass = []
for p in D:
    d1 = distance_squared(p, c1)
    d2 = distance_squared(p, c2)
    print(f"El punto {p} está a distancia {d1} de c1 y a distancia {d2} de c2")
    g = min([(d1, 1), (d2, 2)])[1]
    print(f"El grupo asignado a {p} es {g}")
    print()
    ass.append(g)

Y de nuevo, volvemos a mejorar los centros...

In [ ]:
n1 = ass.count(1)
c1 = [
    sum(x for i, (x, _) in enumerate(D) if ass[i] == 1) / n1,
    sum(x for i, (_, x) in enumerate(D) if ass[i] == 1) / n1,
]

n2 = ass.count(2)
c2 = [
    sum(x for i, (x, _) in enumerate(D) if ass[i] == 2) / n2,
    sum(x for i, (_, x) in enumerate(D) if ass[i] == 2) / n2,
]

c1, c2

Va de nuez, ¿qué asignaciones obtenemos con estos centros mejorados?

In [ ]:
ass = []
for p in D:
    d1 = distance_squared(p, c1)
    d2 = distance_squared(p, c2)
    print(f"El punto {p} está a distancia {d1} de c1 y a distancia {d2} de c2")
    g = min([(d1, 1), (d2, 2)])[1]
    print(f"El grupo asignado a {p} es {g}")
    print()
    ass.append(g)

¡Oh caracoles! la asignación quedó igualita, ejemplos 1 y 2 asignado a grupo 1, ejemplos 3 y 4 asignados a grupo 2. El proceso convergió y el método de $K$-medias ha concluido. Nuestro conjunto de datos se encuentra ahora agrupado.

## Sobre la distancia cuadrada y la eficiencia del método

Sean $\phi(x_i)$ las características de el $i$-ésimo ejemplo de nuestro conjunto de datos y sea $c$ algún centro. Consideremos que ambos vectores consisten de $n$ componentes. La distancia cuadrada entre estos dos puntos se calcula de la siguiente manera denotando como $\phi(x_i)_k$ y $c_k$ a la $k$-ésima componente de $\phi(x_i)$ y $c$ respectivamente:

$$\begin{aligned}
\Vert \phi(x_i) - c_j \Vert^2 &= \left(\sqrt{\sum_{k=1}^n (\phi(x_i)_k - c_k)^2}\right)^2 \\
&= \sum_{k=1}^n (\phi(x_i)_k - c_k)^2
\end{aligned}$$

Utilizamos esta igualdad para realizar los cálculos de arriba. Sin embargo, este cálculo puede ser muy costoso de realizar en cada época si tenemos muchos ejemplos, especialmente si utilizamos representaciones dispersas de los vectores.

Cuando la máquina que utilizamos cuenta con suficiente memoria, podemos sacrificar un poco de espacio para hacer más eficiente el cálculo de las distancias cuadradas de la siguiente manera. Observemos que:

$$\begin{aligned}
\sum_{k=1}^n (\phi(x_i)_k - c_k)^2 &= \sum_{k=1}^n (\phi(x_i)_k^2 -2\phi(x_i)_kc_k + c_k^2) \\
&= \sum_{k=1}^n \phi(x_i)_k^2 -2\sum_{k=1}^n\phi(x_i)_kc_k + \sum_{k=1}^n c_k^2 \\
&= \phi(x_i)\cdot\phi(x_i) + c\cdot c - 2\phi(x_i)\cdot c
\end{aligned}$$

Nota que el término $\phi(x_i)\cdot\phi(x_i)$ es independiente de la época, podemos precalcular estos productos puntos al inicio de todo el método.

Nota que el término $c\cdot c$ depende de la época, pero no del ejemplo con el cuál calculamos la distancia, podemos precalcular estos productos puntos al inicio de cada época.

Nota que el término $- 2\phi(x_i)\cdot c$ depende tanto de la época, como del ejemplo y centro para el cuál queremos calcular la distancia. No podemos deshacernos de esta parte del cálculo. Sin embargo, piensa en qué ocurre si los vectores son dispersos: En la ecuación original debíamos contemplar la **unión** de las dimensiones del ejemplo y del centro en cuestión, pero ahora este término solo requiere contemplar la **intersección** de las dimensiones del ejemplo y del centro en cuestión.

# Componentes de los algoritmos de aprendizaje

- **Conjuntos de datos:** Un contenedor con todos los datos de nuestro problema.
- **Modelos:** Objetos que nos representen posibles predictores
- **Pérdida:** Objetos funcionales que determinen que tan mal predice los datos un modelo.
- **Algoritmos de optimización:** Encuentra el modelo que minimiza una pérdida.

## Conjuntos de datos

- `inputs` como valor obligatorio, serían nuestras $x$s
- `outputs` como valor opcional, regresión y clasificación la necesitan pero $K$-medias no, serían las $y$s
- `extract` una extractor de características, nuestra $\phi$

In [ ]:
def identity(x):
    return x

class Dataset:
    def __init__(self, inputs, outputs = None, extract = identity):
        self.inputs = list(inputs)
        self.outputs = None if outputs is None else list(outputs)
        self.extract = extract
        self.features = [extract(input) for input in self.inputs]

### El que usamos en regresión

In [ ]:
    # (1, 1),
    # (2, 3),
    # (4, 3),
ds1 = Dataset(
    inputs = [1, 2, 4],
    outputs = [1, 3, 3],
    extract = lambda x: [x],
)

### El que usamos en clasificación

In [ ]:
    # (( 0,  2), +1),
    # ((-2,  0), +1),
    # (( 1, -1), -1),
    # ((-1, 1),  -1),
ds2 = Dataset(
    inputs = [[0, 2], [-2, 0], [1, -1], [-1, 1]],
    outputs = [+1, +1, -1, -1],
    extract = lambda x: {"altura": x[0], "peso": x[1]}
)

### El que usamos en descenso de gradiente estocástico

In [ ]:
import numpy as np

In [ ]:
trueW = np.array([1, 2, 3, 4, 5])

In [ ]:
def random_feature_maker(n):
    def random_feature():
        return np.random.randn(n)
    return random_feature

def random_output_maker(real_weights):
    def random_output(x):
        return real_weights.dot(x) + np.random.randn()
    return random_output

class DummyDataset(Dataset):
    def __init__(self, random_feature, random_output, examples = 10**6):
        inputs = [random_feature() for _ in range(examples)]
        outputs = [random_output(x) for x in inputs]
        super().__init__(inputs, outputs)

In [ ]:
ds3 = DummyDataset(
    examples = 10,
    random_feature = random_feature_maker(len(trueW)),
    random_output = random_output_maker(trueW),
)

### El que usamos en clasificación de dígitos

In [ ]:
import requests
import gzip
import os
import hashlib

In [ ]:
def fetch(url):
    hash = hashlib.md5(url.encode("utf-8")).hexdigest()
    path = os.path.join(".", hash)
    if os.path.isfile(path):
        with open(path, "rb") as f:
            data = f.read()
    else:
        with open(path, "wb") as f:
            data = requests.get(url).content
            f.write(data)
    return np.frombuffer(
        gzip.decompress(data),
        dtype=np.uint8,
    ).copy()

In [ ]:
def fetch_from_url(images_url, labels_url):
    def load_data():
        image_data = fetch(images_url)
        label_data = fetch(labels_url)
        return image_data, label_data
    return load_data

class MNISTDataset(Dataset):
    def __init__(self, load_data):
        image_data, label_data = load_data()
        
        # Validate image_data
        [image_magic] = image_data[0:4][::-1].copy().view(np.uint32)
        assert image_magic == 2051
        [image_total] = image_data[4:8][::-1].copy().view(np.uint32)
        [image_rows, image_cols] = image_data[8:16][::-1].copy().view(np.uint32)
        assert len(image_data[16:]) == image_total * image_rows * image_cols
        images = image_data[16:].reshape((image_total, image_rows, image_cols))

        # Validate label_data
        [label_magic] = label_data[0:4][::-1].copy().view(np.uint32)
        assert label_magic == 2049
        [label_total] = label_data[4:8][::-1].copy().view(np.uint32)
        assert label_total == image_total
        assert len(label_data[8:]) == label_total
        labels = label_data[8:]

        super().__init__(
            inputs = list(images),
            outputs = list(labels),
            extract = lambda image: image.reshape(image_rows * image_cols),
        )

In [ ]:
ds4 = MNISTDataset(
    load_data=fetch_from_url(
        "http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz",
        "http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz",
    ),
)

In [ ]:
class MNISTDatasetBinary(MNISTDataset):
    def __init__(self, load_data, digit):
        super().__init__(load_data)
        self.outputs = [+1 if output == digit else -1 for output in self.outputs]

In [ ]:
ds5 = MNISTDatasetBinary(
    load_data=fetch_from_url(
        "http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz",
        "http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz",
    ),
    digit = 0,
)

### El que usamos en optimización equitativa de grupos

In [ ]:
class GroupsDataset(Dataset):
    def __init__(self, inputs, outputs, groups, extract = identity):
        assert len(inputs) == len(outputs) == len(groups)
        self.groups = groups
        super().__init__(inputs, outputs, extract)

In [ ]:
    # (1, 4, 'A'),
    # (2, 8, 'A'),
    # (5, 5, 'B'),
    # (6, 6, 'B'),
    # (7, 7, 'B'),
    # (8, 8, 'B'),
ds6 = GroupsDataset(
    inputs = [1, 2, 5, 6, 7, 8],
    outputs = [4, 8, 5, 6, 7, 8],
    groups = ['A', 'A', 'B', 'B', 'B', 'B'],
    extract = lambda x: [x],
)

### Extractores de características no lineales

In [ ]:
ds7 = Dataset(
    inputs = np.array([
        1.87270059, 4.75357153, 3.65996971, 2.99329242, 0.7800932 ,
        0.7799726 , 0.29041806, 4.33088073, 3.00557506, 3.54036289,
        0.10292247, 4.84954926, 4.1622132 , 1.06169555, 0.90912484,
        0.91702255, 1.52121121, 2.62378216, 2.15972509, 1.4561457,
    ]),
    outputs = np.array([
        3.28315199, 1.87377693, 2.7730387 , 3.06769436, 2.61445411,
        2.94347711, 1.97322331, 2.59380959, 3.29129334, 2.57997942,
        2.20834872, 1.81644778, 2.26246105, 3.2851416 , 3.20945528,
        3.05723383, 2.86300827, 2.84460771, 3.41107562, 2.97222613,
    ]),
    extract = lambda x: np.array([1, x, x ** 2]),
)

### $K$-medias

In [ ]:
ds8 = Dataset(
    inputs = ((1, 3), (2, 3),
              (5, 3), (7, 3)),
    extract = lambda x: list(x),
)

## Modelos

In [ ]:
from abc import ABC, abstractmethod

In [ ]:
class Model(ABC):
    @abstractmethod
    def predict(self, features):
        ...

### Regresión y clasificación lineal

In [ ]:
class LinearModel(Model):
    def __init__(self, weights):
        self.weights = weights

    @abstractmethod
    def combine(self, weights, features):
        ...

    def activate(self, score):
        return score

    def predict(self, features):
        return self.activate(self.combine(self.weights, features))

In [ ]:
class LinearPredictor(LinearModel):
    def combine(self, weights, features):
        return sum(w * x for w, x in zip(weights, features))

In [ ]:
class LinearPredictorSparse(LinearModel):
    def combine(self, weights, features):
        if len(weights) < len(features):
            features, weights = weights, features
        return sum(weights.get(i, 0) * x for i, x in features.items())

In [ ]:
class LinearClassifier(LinearPredictor):
    def activate(self, score):
        if score >= 0:
            return +1
        return -1

In [ ]:
class LinearClassifierSparse(LinearPredictorSparse, LinearClassifier):
    pass

### Clusters

In [ ]:
class Clusters(Model):
    def __init__(self, centers, distance):
        self.centers = centers
        self.distance = distance

    def predict(self, features):
        return min(
            (self.distance(features, c), i) 
            for i, c
            in enumerate(self.centers)
        )